In [76]:
import time
import os
import random
from datetime import date
import datetime
import pandas as pd
import numpy as np
from dash import Dash, dcc, html,Input, Output
import plotly.graph_objects as go

def elimina_registros(fecha):
    
    if fecha.month<10:
        mes_actual= str("0")+str(fecha.month)
    else: mes_actual = str(fecha.month)
    
    anio_actual=str(fecha.year)
    dict_mes={"01":"Ene","02":"Feb","03":"Mar","04":"Abr","05":"May","06":"Jun","07":"Jul","08":"Ago","09":"Sep","10":"Oct","11":"Nov","12":"Dic"}
    mes_actual2=dict_mes[mes_actual]
    nombre_consolidado=  anio_actual+mes_actual + " " + "-" + " " + "Reporte Servicios" + " " + mes_actual2 + " " + anio_actual[2:4] + ".xlsx"
    ruta=os.path.join(r"C:\Users\nathalia.gutierrez\Grupo Socofar\Nicolas Chavarro Santana - Dashboard OMS\Querys result\Última Milla",nombre_consolidado)
    
    
    ayer =  fecha - datetime.timedelta(days=1)
    dfTotal = pd.read_excel(ruta)
    mesdfTotal=dfTotal["Mes"]
    diadfTotal=dfTotal["Dia"]
    aux=0
    for dato in mesdfTotal:
        print(str(dato)+ "-" +str(diadfTotal[aux]))
        if  dato==fecha.month and diadfTotal[aux]==fecha.day:
            print("Registro " + " " + str(aux) + " " +"eliminado del archivo consolidado")
            dfTotal=dfTotal.drop(aux)
        else: print("el registro"  + " " + str(aux) + " "+  "no se debe eliminar")
        aux=aux+1
    
    aux=0
    for dato2 in mesdfTotal:
        print(str(dato2)+ "-" +str(diadfTotal[aux]))
        if  dato2==ayer.month and diadfTotal[aux]==ayer.day:
            print("Registro " + " " + str(aux) + " " +"eliminado del archivo consolidado")
            dfTotal=dfTotal.drop(aux)
        else: print("el registro"  + " " + str(aux) + " "+  "no se debe eliminar")
        aux=aux+1
    
    return dfTotal

def edicion_quick(fecha,dfTotal):
    dfQuick = pd.read_csv(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick\QuickActual.csv",sep = ";")
    
    NewColumns= ['ID Servicio', 'Valor Declarado', 'Ida y Vuelta', 'Total (Km)',
       'Precio Total', 'Número de orden', 'Precio Base', 'Fecha de solicitud',
       'Fecha de creación', 'ID Empresa', 'Ciudad', 'Servicio',
       'Tipo de Servicio', 'Nombre', 'Tipo de documento',
       'Tipo de identificación', 'N. Identificación', 'Dirección', 'Teléfono',
       'Email', 'Recargos', 'Nombre Trabajador', 'Empresa', 'Placa',
       'Cantidad Paradas', 'Estado', 'Origen', 'Dirección de Origen',
       'Detalle', 'Recargp KM', 'Recargo Min', 'Recargo Paradas',
       'Recargo Valor Declarado', 'Recargo Periferia', 'Recargo Nocturo',
       'Recargo Dominical', 'Propina', 'Fecha Hora Asignado',
       'Tiempo Asignación', 'Fecha Hora Llegada Primera Parada',
       'Trayecto Primera Parada', 'Tiempo Primera Parada',
       'Fecha Hora Salida Primera Parada', 'Tiempo Salida Primera Parada',
       'Fecha Hora Llegada Segunda Parada', 'Tiempo Llegada Segunda Parada',
       'Fecha Hora Salida Segunda Parada', 'Tiempo Salida Segunda Parada',
       'Tiempo Llegada Tercera Parada', 'Tiempo Salida Tercera Parada',
       'Fecha Hora Finalizacion Servicio', 'Tiempo Finalizacion',
       'Descripcion', 'Relanzado', 'Novedad', 'Tipo de Novedad']
    

    
        
    dfQuick["Fecha de solicitud"] = pd.to_datetime(dfQuick["Fecha de solicitud"],format="%Y/%m/%d %H:%M:%S")
    dfQuick["Fecha Hora Asignado"] = pd.to_datetime(dfQuick["Fecha Hora Asignado"],format="%Y/%m/%d %H:%M:%S")
    dfQuick["Fecha Hora Llegada Primera Parada"] = pd.to_datetime(dfQuick["Fecha Hora Llegada Primera Parada"],format="%Y/%m/%d %H:%M:%S")
    dfQuick["Fecha Hora Salida Primera Parada"] = pd.to_datetime(dfQuick["Fecha Hora Salida Primera Parada"],format="%Y/%m/%d %H:%M:%S")
    dfQuick["Fecha Hora Llegada Segunda Parada"] = pd.to_datetime(dfQuick["Fecha Hora Llegada Segunda Parada"],format="%Y/%m/%d %H:%M:%S")
    dfQuick=dfQuick[dfQuick["Email"]=="servicios.oms@cruzverde.com.co"]
    
    
    dfQuick= dfQuick[['ID Servicio', 'Total (Km)',
       'Precio Total', 'Número de orden', 'Fecha de solicitud',
       'Email','Estado', 'Fecha Hora Asignado','Fecha Hora Llegada Primera Parada',
       'Fecha Hora Salida Primera Parada','Fecha Hora Llegada Segunda Parada','Fecha Hora Salida Segunda Parada', 'Tipo de Novedad']]
    
    dfQuick=dfQuick.reindex(columns=["Número de orden","Fecha de solicitud","Fecha Hora Asignado","Fecha Hora Llegada Primera Parada","Fecha Hora Salida Primera Parada","Fecha Hora Llegada Segunda Parada","Fecha Hora Salida Segunda Parada","Total (Km)","Tipo de Novedad","Precio Total","Estado"])
    
    dfQuick.insert(8,"Finalizado Fallido","No",allow_duplicates=True)
    dfQuick.insert(11,"Proveedor","Quick",allow_duplicates=True)
    dfQuick["Tipo Fallido"] = dfQuick["Tipo de Novedad"]
    dfQuick = dfQuick.drop(["Tipo de Novedad"],axis=1)
    dfQuick.columns=["numorden","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","distancia_km","Finalizado Fallido","Valor Servicio","Proveedor","Estado UM","Tipo Fallido"] 
    dfQuick.insert(13,"Mes",fecha.month,allow_duplicates=True)
    dfQuick.insert(14,"Dia",fecha.day,allow_duplicates=True)
    dfQuick = dfQuick.reindex(columns=["numorden","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","distancia_km","Finalizado Fallido","Tipo Fallido","Valor Servicio","Proveedor","Estado UM","Mes","Dia"])
    
    dfQuick['iniciado'] = pd.to_datetime(dfQuick.iniciado).dt.tz_localize(None)
    dfQuick['asignado'] = pd.to_datetime(dfQuick.asignado).dt.tz_localize(None)
    dfQuick['llego_punto'] = pd.to_datetime(dfQuick.llego_punto).dt.tz_localize(None)
    dfQuick['salio_punto'] = pd.to_datetime(dfQuick.salio_punto).dt.tz_localize(None)
    dfQuick['llego_cliente'] = pd.to_datetime(dfQuick.llego_cliente).dt.tz_localize(None)
    dfQuick['finalizado'] = pd.to_datetime(dfQuick.finalizado).dt.tz_localize(None)
    
    
    listainiciado=dfQuick["iniciado"]
    ayer =  fecha - datetime.timedelta(days=1)
    
    aux=0
    for elemento in listainiciado:
        diaServ= elemento.day
        if diaServ == fecha.day:
            dfQuick.iloc[aux,14]= fecha.day
        elif diaServ == ayer.day:
            dfQuick.iloc[aux,14] = ayer.day
        else: dfQuick.iloc[aux,14] = dfQuick.iloc[aux,14]
        aux=aux+1
    
    
    dfQuick.loc[dfQuick["Tipo Fallido"].isnull()==False,"Finalizado Fallido"]="Si"
    
    dfTotal = pd.concat([dfTotal,dfQuick],axis=0)
    
    for the_file in os.listdir(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick"):
        if the_file =="QuickActual.xls":
            nombre_archivo=os.path.join(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick",the_file)
            os.rename(nombre_archivo,r"C:\Users\nathalia.gutierrez\Downloads\QuickActual.xlsx")
            
    return dfTotal



def edicion_mu(fecha,dfTotal):
    
    for the_file in os.listdir(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick"):
        if "query_result" in the_file:
            nombre_archivo=os.path.join(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick",the_file)
    os.rename(nombre_archivo,r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick\MUActual.csv")
    
    dfMU = pd.read_csv(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick\MUActual.csv")
    
    dfMU.columns = ["uuid","num_orden","date_created","fecha_inicio","hora_inicio","resource_name","valor_total_servicio","valor_descuento","recargo_seguro","recargo_distancia","recargo_ida_vuelta","recargo_tiempo","recargo_nocturno","recargo_parqueadero","recargo_paradas","valor_base","ida_vuelta","tipo_pago","cant_paradas","distancia_km","tiempo_adicional","estado_servicio","descripcion","dir_origen_lupap","dir_destino_lupap","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","tiempo_asignacion","tiempo_recogida","tiempo_punto","tiempo_entrega","tiempo_cliente","total_productos","entrega_fallida","tipificación_fallida","comentario_fallida","motivo_finalizado","id_solicitante","nombre_solicitante","nombre_empresa","ciudad"]
    
    dfMU=dfMU.drop(["uuid","date_created","resource_name","valor_descuento","recargo_seguro","recargo_distancia","recargo_ida_vuelta","recargo_tiempo","recargo_nocturno","recargo_parqueadero","recargo_paradas","valor_base","ida_vuelta","tipo_pago","cant_paradas","tiempo_adicional","descripcion","dir_origen_lupap","dir_destino_lupap","tiempo_asignacion","tiempo_recogida","tiempo_punto","tiempo_entrega","tiempo_cliente","total_productos","tipificación_fallida","comentario_fallida","motivo_finalizado","id_solicitante","nombre_empresa","ciudad"],axis=1)
    dfMU=dfMU[dfMU.nombre_solicitante.isin(["OMS Cruz Verde CO", "Back Office Call Center Interactivo OMS","Back Office Call Center Konecta Cruz Verde OMS"])]
    
    Referencia=dfMU["hora_inicio"]
    aux=0

    for l in Referencia:
        hora = l[0:8]
        fecha2 = dfMU.iloc[aux,1]
        fecha2 = fecha2[0:10]
        dfMU.iloc[aux,1]= str(fecha2) + " " + str(hora)
        aux=aux+1
    
           
    dfMU["fecha_inicio"] = pd.to_datetime(dfMU["fecha_inicio"],format="%Y/%m/%d %H:%M:%S")
    
    dfMU=dfMU.drop(["nombre_solicitante","iniciado","hora_inicio"],axis=1)
    dfMU=dfMU.reindex(columns=["num_orden","fecha_inicio","asignado","llego_punto","salio_punto","llego_cliente","finalizado","distancia_km","entrega_fallida","tipificación_fallida","valor_total_servicio","estado_servicio"])
    dfMU.columns=["numorden","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","distancia_km","Finalizado Fallido","Tipo Fallido","Valor Servicio","Estado UM"]
    dfMU.insert(11,"Proveedor","MU",allow_duplicates=True)
    dfMU.insert(13,"Mes",fecha.month,allow_duplicates=True)
    dfMU.insert(14,"Dia",fecha.day,allow_duplicates=True)
    
    
    
    dfMU['asignado'] = pd.to_datetime(dfMU.asignado).dt.tz_localize(None)
    dfMU['llego_punto'] = pd.to_datetime(dfMU.llego_punto).dt.tz_localize(None)
    dfMU['salio_punto'] = pd.to_datetime(dfMU.salio_punto).dt.tz_localize(None)
    dfMU['llego_cliente'] = pd.to_datetime(dfMU.llego_cliente).dt.tz_localize(None)
    dfMU['finalizado'] = pd.to_datetime(dfMU.finalizado).dt.tz_localize(None)
    
    dfMU["asignado"] = pd.to_datetime(dfMU["asignado"],format="%Y/%m/%d %H:%M:%S")
    dfMU["llego_punto"] = pd.to_datetime(dfMU["llego_punto"],format="%Y/%m/%d %H:%M:%S")
    dfMU["salio_punto"] = pd.to_datetime(dfMU["salio_punto"],format="%Y/%m/%d %H:%M:%S")
    dfMU["llego_cliente"] = pd.to_datetime(dfMU["llego_cliente"],format="%Y/%m/%d %H:%M:%S")
    dfMU["finalizado"] = pd.to_datetime(dfMU["finalizado"],format="%Y/%m/%d %H:%M:%S")
    
    listainiciado=dfMU["iniciado"]
    
    ayer =  fecha - datetime.timedelta(days=1)
    aux=0
    for elemento in listainiciado:
        diaServ= elemento.day
        if diaServ == fecha.day:
            dfMU.iloc[aux,14]= fecha.day
        elif diaServ == ayer.day:
            dfMU.iloc[aux,14] = ayer.day
        else: dfMU.iloc[aux,14] = dfMU.iloc[aux,14]
        aux=aux+1
    
    for the_file in os.listdir(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick"):
        if the_file =="MUActual.xlsx":
            nombre_archivo=os.path.join(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick",the_file)
            os.rename(nombre_archivo,r"C:\Users\nathalia.gutierrez\Downloads\MUActual.xlsx")
    
    dfTotal=pd.concat([dfTotal,dfMU],axis=0)
    
    return dfTotal


def edicion_picap(fecha,dfTotal):
    dfPicap = pd.read_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick\Reporte.xlsx")
    dfPicap=dfPicap.drop(["ID","Fecha de creación","Hora de creación","Fecha de programación","Dirección de recogida","Complemento dirección de recogida","Dirección de entrega","Complemento dirección de entrega","Indicaciones","Valor declarado","Pago contra entrega","Razón de la no colecta","Nombre del piloto","Teléfono del piloto","ID del booking","Costo estimado","Uso de billetera","Tiempo tomado desde creado el pedido hasta recogido (Minutos)","Tiempo tomado desde recogido hasta entregado (Minutos)","Tiempo de espera desde la llegada hasta recogida (Minutos)","Conductor requería una base de dinero","Agente que canceló","Evidencia de recogida","Evidencia de entrega","Ida y vuelta"],axis=1)
    dfPicap = dfPicap[~dfPicap["Hora de programación"].isnull()]
    #Referencia=dfPicap["Hora de programación"]
    #aux=0

    #for l in Referencia:
       # if  'nan' in str(l):
            #dfPicap.iloc[aux,0] = np.nan
       # else:
            #dfPicap.iloc[aux,0]= str(dfPicap.iloc[aux,0]) + " " + str(l)
        #aux=aux+1
        
    dfPicap["Hora de programación"] = pd.to_datetime(dfPicap["Hora de programación"],format="%Y-%m-%d %I:%M:%S %p")
    dfPicap["Hora de aceptación"] = pd.to_datetime(dfPicap["Hora de aceptación"],format="%Y-%m-%d %I:%M:%S %p")
    dfPicap["Hora de llegada a recogida"] = pd.to_datetime(dfPicap["Hora de llegada a recogida"],format="%Y-%m-%d %I:%M:%S %p")
    dfPicap["Hora de recogida"] = pd.to_datetime(dfPicap["Hora de recogida"],format="%Y-%m-%d %I:%M:%S %p")
    dfPicap["Hora de entrega"] = pd.to_datetime(dfPicap["Hora de entrega"],format="%Y-%m-%d %I:%M:%S %p")
    dfPicap=dfPicap.reindex(columns=["Número de pedido","Hora de programación" ,"Hora de aceptación","Hora de llegada a recogida","Hora de recogida","Hora de entrega","Distancia (KM)","Razón de la no entrega","Costo final","Estado","Número de paradas"])
    dfPicap=dfPicap.drop(["Número de paradas"],axis=1)
    dfPicap.columns=["numorden","iniciado","asignado","llego_punto","salio_punto","llego_cliente","distancia_km","Tipo Fallido","Valor Servicio","Estado UM"]
    
    dfPicap.insert(6,"finalizado",dfPicap["llego_cliente"],allow_duplicates=True)
    dfPicap.insert(8,"Finalizado Fallido","No",allow_duplicates=True)
    dfPicap.insert(11,"Proveedor","Picap",allow_duplicates=True)
    dfPicap.insert(13,"Mes",fecha.month,allow_duplicates=True)
    dfPicap.insert(14,"Dia",fecha.day,allow_duplicates=True)
    
    dfPicap["Valor Servicio"] = dfPicap["Valor Servicio"].fillna("$")
    plata = dfPicap["Valor Servicio"]
    plata2 = []
    for registro in plata:
        valorn = registro.replace("$","")
        valorn = valorn.replace(".","")
        valorn = valorn[0:valorn.find(",")]
        plata2.append(valorn)
    
    dfPicap["Valor Servicio"] = plata2
    dfPicap["Valor Servicio"] = pd.to_numeric(dfPicap["Valor Servicio"])
    
    orden_larga = dfPicap["numorden"]
    numorden_v2 = []
    for registro in orden_larga:
        numorden_new = registro[7:len(registro)]
        numorden_v2.append(numorden_new)
        
    dfPicap["numorden"] = numorden_v2
    
    etiqueta_picap = {"Estadofinal":{"Cancelado":"Cancelado",
                                      "Devuelto":"Cancelado",
                                      "Entregado":"Finalizado",
                                      "Esperando recogida":"En espera",
                                      "Recogido":"En transito"}}
    
    dfPicap["Estado UM"] = dfPicap["Estado UM"].map(etiqueta_picap["Estadofinal"]) 
    
    listainiciado=dfPicap["iniciado"]
    ayer =  fecha - datetime.timedelta(days=1)
    
    aux=0
    for elemento in listainiciado:
        diaServ= elemento.day
        if diaServ == fecha.day:
            dfPicap.iloc[aux,14]= fecha.day
        elif diaServ == ayer.day:
            dfPicap.iloc[aux,14] = ayer.day
        else: dfPicap.iloc[aux,14] = dfPicap.iloc[aux,14]
        aux=aux+1
    dfPicap.loc[dfPicap["Tipo Fallido"].isnull()==False,"Finalizado Fallido"]="Si"
    
    dfTotal=pd.concat([dfTotal,dfPicap],axis=0)
    
    return dfTotal

def edicion_alianza(fecha,dfTotal):
    df_alianza = pd.read_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\DescargaQuick\domicilios.xlsx")
    df_alianza = df_alianza.drop(["id","Pedido","Origen","Ciudad","Punto","Mensajero","Fecha","Fecha Pedido","Hora Pedido","Fecha Asignación","Fecha Recogida","Fecha Entrega","Trans. Factura","Trans. Asignación","Trans. Recogida","Trans. Entrega","Trans. Total","Coordinador","Lat Entrega","Lon Entrega","Num. Orden Cruz Verde","No. de Viajes","Destino 1","Destino 2","Destino 3","Destino 4","Destino 5"],axis = 1)
    df_alianza = df_alianza.loc[(df_alianza["Factura"].str.contains("CO",regex = True)) | (df_alianza["Factura"].str.contains("COCC",regex=True))]
    
    j = 1
    for elemento in ["Hora Factura","Hora Asignación","Hora Recogida","Hora Entrega"]:
        i = 0
        for elemento2 in df_alianza[elemento]:
            if elemento2 == " ":
                df_alianza.iloc[i,j] = np.nan
            
            i = i +1 
        j = j + 1
        
    
    df_alianza["Hora Factura"] = pd.to_datetime(df_alianza["Hora Factura"], format = "%Y-%m-%d %H:%M:%S" )
    df_alianza["Hora Asignación"] = pd.to_datetime(df_alianza["Hora Asignación"], format = "%Y-%m-%d %H:%M:%S" )
    df_alianza["Hora Recogida"] = pd.to_datetime(df_alianza["Hora Recogida"], format = "%Y-%m-%d %H:%M:%S" )
    df_alianza["Hora Entrega"] = pd.to_datetime(df_alianza["Hora Entrega"], format = "%Y-%m-%d %H:%M:%S" )
    
    df_alianza["Valor Servicio"] = np.nan
    df_alianza["Proveedor"] = "Alianza"
    df_alianza["Finalizado Fallido"] = "No"
    df_alianza["salio_punto"] = np.nan
    df_alianza["finalizado"] = df_alianza["Hora Entrega"]
    df_alianza.loc[~df_alianza.Novedad.isnull(),"Finalizado Fallido"] = "Si"
    df_alianza["Mes"] = fecha.month
    df_alianza["Dia"] = fecha.day

    listainiciado=df_alianza["Hora Factura"]
    ayer =  fecha - datetime.timedelta(days=1)
    
    aux=0
    for elemento in listainiciado:
        diaServ= elemento.day
        if diaServ == fecha.day:
            df_alianza.iloc[aux,14]= fecha.day
        elif diaServ == ayer.day:
            df_alianza.iloc[aux,14] = ayer.day
        else: df_alianza.iloc[aux,14] = df_alianza.iloc[aux,14]
        aux=aux+1

    
    
    etiqueta_alianza = {"Estadofinal":{"SOLICITADO":"En Espera",
                                      "NOVEDAD":"Cancelado - Finalizado Fallido",
                                      "ENTREGADO":"Finalizado",
                                      "ASIGNADO":"En Proceso",
                                      "RECOGIDO":"En Transito"}}
    
    df_alianza["Estado"] = df_alianza["Estado"].map(etiqueta_alianza["Estadofinal"])

    df_alianza=df_alianza.reindex(columns = ["Factura","Hora Factura","Hora Asignación","Hora Recogida","salio_punto","Hora Entrega","finalizado","Distancia (KM)","Finalizado Fallido","Novedad","Valor Servicio","Proveedor","Estado","Mes","Dia"])
    df_alianza.loc[(df_alianza["Estado"] == "Cancelado - Finalizado Fallido")&(df_alianza["Novedad"] == "CANCELADO"),"Estado"] ="Cancelado" 
    df_alianza.loc[(df_alianza["Estado"] == "Cancelado - Finalizado Fallido")&(df_alianza["Novedad"] != "CANCELADO"),"Estado"] ="Finalizado" 

    df_alianza.columns = ["numorden","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","distancia_km","Finalizado Fallido","Tipo Fallido","Valor Servicio","Proveedor","Estado UM","Mes","Dia"]
    
    dfTotal=pd.concat([dfTotal,df_alianza],axis=0)
    
    return dfTotal
    
    



fecha = date.today()
Consolidado = elimina_registros(fecha)
Consolidado = edicion_quick(fecha,Consolidado)
Consolidado = edicion_mu(fecha,Consolidado)
Consolidado = edicion_picap(fecha,Consolidado)
Consolidado = edicion_alianza(fecha,Consolidado)


if fecha.month<10:
    mes_actual= str("0")+str(fecha.month)
else: mes_actual = str(fecha.month)


anio_actual=str(fecha.year)
dict_mes={"01":"Ene","02":"Feb","03":"Mar","04":"Abr","05":"May","06":"Jun","07":"Jul","08":"Ago","09":"Sep","10":"Oct","11":"Nov","12":"Dic"}
mes_actual2=dict_mes[mes_actual]
nombre_consolidado=  anio_actual+mes_actual + " " + "-" + " " + "Reporte Servicios" + " " + mes_actual2 + " " + anio_actual[2:4] + ".xlsx"
ruta=os.path.join(r"C:\Users\nathalia.gutierrez\Grupo Socofar\Nicolas Chavarro Santana - Dashboard OMS\Querys result\Última Milla",nombre_consolidado)
Consolidado.to_excel(ruta,index=False,sheet_name="Hoja1")

In [77]:
fecha = date.today()
anio_actual=str(fecha.year)
if fecha.month<10:
    mes_actual= str("0")+str(fecha.month)
else: mes_actual = str(fecha.month)

dict_mes={"01":"Ene","02":"Feb","03":"Mar","04":"Abr","05":"May","06":"Jun","07":"Jul","08":"Ago","09":"Sep","10":"Oct","11":"Nov","12":"Dic"}
mes_actual2=dict_mes[mes_actual]

tiempos_total = pd.DataFrame(columns=["ORDER_NO","STATUS","STATUS_DATE","SHIPNODE_KEY"])
for the_file in os.listdir(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\TiemposOMS"):
    archivo_subida = os.path.join(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\TiemposOMS",the_file) ###Importación de la base de tiempos
    tiempos_parcial=pd.read_csv(archivo_subida,sep=",")
    columnas_tiempos= ["RN","ORDER_NO","ORDER_HEADER_KEY","STATUS","STATUS_DATE","CREATEUSERID","MODIFYUSERID","SHIPMENT_KEY","SCAC","SHIPNODE_KEY","ASSIGNED_TO_USER_ID","BLANCOI","BLANCOII","BLANCOIII","BLANCOIV","BLANCOV","BLANCOVI","BLANCOVII","BLANCOVIII","BLANCOIX","BLANCOX","BLANCOXI","BLANCOXII","BLANCOXIII"]  
    tiempos_parcial.columns=columnas_tiempos
    tiempos_parcial=tiempos_parcial.drop(0,axis=0)
    print(the_file)
    tiempos_parcial = tiempos_parcial[["RN","ORDER_NO","STATUS","STATUS_DATE","SHIPNODE_KEY"]]
    tiempos_total = pd.concat([tiempos_total,tiempos_parcial],axis=0)

resumen_total = pd.DataFrame(columns=["ORDER_NO","ORDER_DATE","LEVEL_OF_SERVICE","EXTN_ORG_REQ_SHIP_DATE","SHIPNODE_KEY"])
for the_file in os.listdir(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Resumen OMS"):
    archivo_subida = os.path.join(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Resumen OMS",the_file) ###Importación de la base de tiempos
    resumen_parcial=pd.read_csv(archivo_subida,sep=",")
    columnas_resumen= ["RN","ORDER_HEADER_KEY","ENTRY_TYPE","ORDER_NO","DOCUMENT_TYPE","ORDER_DATE","ORDER_TYPE","ENTRY_TYPE2","LEVEL_OF_SERVICE","CUSTOMER_PHONE_NO","EXTN_RUN_RUT_NIT","ORIGINAL_TOTAL_AMOUNT","ENTERPRISE_KEY","PAYMENT_TYPE","STATUS_NAME","EXTN_ORG_REQ_SHIP_DATE","STATUS_DATE","EXTN_ET_FULFILMENT","SHIPNODE_KEY","BLANCOI","BLANCOII","BLANCOIII","BLANCOIV"]  
    resumen_parcial.columns=columnas_resumen
    resumen_parcial=resumen_parcial.drop(0,axis=0)
    print(the_file)
    resumen_parcial = resumen_parcial[["ORDER_NO","ORDER_DATE","LEVEL_OF_SERVICE","EXTN_ORG_REQ_SHIP_DATE","SHIPNODE_KEY"]]
    resumen_total = pd.concat([resumen_total,resumen_parcial],axis=0)

    
resumen_total=resumen_total[~resumen_total.EXTN_ORG_REQ_SHIP_DATE.isnull()]

SLAs =  resumen_total["EXTN_ORG_REQ_SHIP_DATE"]
nuevos_sla = []
for elemento in SLAs:
    elemento = elemento[0:10] + " "+ elemento[elemento.find("T")+1:elemento.find("T")+9]
    nuevos_sla.append(elemento)
    
resumen_total["EXTN_ORG_REQ_SHIP_DATE"] = nuevos_sla

resumen_total["EXTN_ORG_REQ_SHIP_DATE"] = pd.to_datetime(resumen_total["EXTN_ORG_REQ_SHIP_DATE"],format ="%Y-%m-%d %H:%M:%S")




fechas_creacion = resumen_total["ORDER_DATE"]
nuevas_fechas_creacion = []
for elemento in fechas_creacion:
    elemento = elemento[0:20]
    nuevas_fechas_creacion.append(elemento)
    

resumen_total["ORDER_DATE"] = nuevas_fechas_creacion
resumen_total["ORDER_DATE"] = pd.to_datetime(resumen_total["ORDER_DATE"],format ="%Y-%m-%d %H:%M:%S")

    
fecha_inicial = resumen_total["EXTN_ORG_REQ_SHIP_DATE"]
fecha_col = []

for elemento in fecha_inicial:
    resta = elemento - datetime.timedelta(hours = 5)
    fecha_col.append(resta)
    
resumen_total["EXTN_ORG_REQ_SHIP_DATE"] = fecha_col

fecha_crea_inicial = resumen_total["ORDER_DATE"]
fecha_crea_col = []
for elemento in fecha_crea_inicial:
    resta2 = elemento - datetime.timedelta(hours = 5)
    fecha_crea_col.append(resta2)
    
resumen_total["ORDER_DATE_COL"] = fecha_crea_col

    

queryResult--2023-3-2-17-8-43_view-only-on-ms-excel.csv
queryResult--2023-3-2-17-9-0_view-only-on-ms-excel.csv
queryResult--2023-3-2-17-9-20_view-only-on-ms-excel.csv
queryResult--2023-3-2-17-9-36_view-only-on-ms-excel.csv
queryResult--2023-3-2-17-9-43_view-only-on-ms-excel.csv
queryResult--2023-3-2-17-5-35_view-only-on-ms-excel.csv


In [78]:
directorio_RD = pd.read_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Directorio_RedDomicilios.xlsx")
resumen_total = pd.merge(resumen_total,directorio_RD,left_on = "SHIPNODE_KEY",right_on ="CodSuc",how = "left")
resumen_total = resumen_total[['ORDER_NO', 'LEVEL_OF_SERVICE', 'EXTN_ORG_REQ_SHIP_DATE',
        'ORDER_DATE_COL', 'CodSuc', 'Ciudad']]

ciudades_inicial = resumen_total["Ciudad"]
ciudad_nueva = []

for elemento in ciudades_inicial:
    
    if (elemento == "Bogotá") | (elemento == "Bogota"):
        ciudad2 = "Bogotá"
    elif (elemento == "Medellin") | (elemento == "Medellín") :
        ciudad2 = "Medellín"
    elif elemento == "Cali":
        ciudad2 = "Cali"
    elif elemento =="Barranquilla":
        ciudad2 = "Barranquilla"
    else: ciudad2 = "Otras ciudades"
    
    ciudad_nueva.append(ciudad2)
    
resumen_total["Ciudad"] = ciudad_nueva
        


In [79]:
tiempos_total_packin = tiempos_total[tiempos_total["STATUS"]=="3350.1500.1000"]

tiempos_total_packin = tiempos_total_packin.sort_values(by=["RN"],ascending=False)
tiempos_total_packin["STATUS_DATE"] = pd.to_datetime(tiempos_total_packin["STATUS_DATE"],format="%Y-%m-%d %H:%M:%S")
status_date_inicial = tiempos_total_packin["STATUS_DATE"]
STATUS_DATE_COL =[]

for fecha in status_date_inicial:                        ###Se convierte el status date a fecha hora colombia
    resta = fecha -datetime.timedelta(hours=5)
    STATUS_DATE_COL.append(resta)
    
    
    
tiempos_total_packin["STATUS_DATE_COL"] = STATUS_DATE_COL

tiempos_total_packin =tiempos_total_packin.drop_duplicates(subset=["ORDER_NO"],keep="first")
resumen_total = resumen_total.drop_duplicates(subset=["ORDER_NO"])


ordenes2=Consolidado["numorden"]
correccion=[]
for element in ordenes2:
    element = str(element)
    correccion.append(element)
Consolidado["numorden"]= correccion

Consolidado_CO = Consolidado[Consolidado.numorden.str.contains("CO1")]
Consolidado_COCC = Consolidado[Consolidado.numorden.str.contains("COCC")]
Consolidado_Y = Consolidado[Consolidado.numorden.str.contains("Y1")]

orden_inicial =Consolidado_CO["numorden"]
numorden2 = []

for elemento in orden_inicial:
    elemento = elemento[elemento.find("C"):elemento.find("C") + 10]
    numorden2.append(elemento)

Consolidado_CO["numorden"] = numorden2


orden_inicial =Consolidado_COCC["numorden"]
numorden2 = []

for elemento in orden_inicial:
    elemento = elemento[elemento.find("C"):elemento.find("C") + 12]
    numorden2.append(elemento)

Consolidado_COCC["numorden"] = numorden2


orden_inicial =Consolidado_Y["numorden"]
numorden2 = []

for elemento in orden_inicial:
    elemento = elemento[elemento.find("Y"):elemento.find("Y") + 10]
    numorden2.append(elemento)

Consolidado_Y["numorden"] = numorden2


Consolidado = pd.concat([Consolidado_CO,Consolidado_COCC,Consolidado_Y],axis = 0)


Consolidado_cruce = pd.merge(Consolidado,tiempos_total_packin,left_on="numorden",right_on="ORDER_NO",how="left") 
Ordenes_sin_3pl = pd.merge(Consolidado,tiempos_total_packin,left_on="numorden",right_on="ORDER_NO",how="right")
Ordenes_sin_3pl = Ordenes_sin_3pl[Ordenes_sin_3pl.numorden.isnull()]
Ordenes_sin_3pl = Ordenes_sin_3pl[["ORDER_NO","SHIPNODE_KEY","STATUS_DATE_COL"]]


Consolidado_cruce = Consolidado_cruce[['numorden', 'iniciado', 'asignado', 'llego_punto', 'salio_punto',
       'llego_cliente', 'finalizado', 'distancia_km', 'Finalizado Fallido',
       'Tipo Fallido', 'Valor Servicio', 'Proveedor', 'Estado UM', 'Mes',
       'Dia', 'STATUS', 'STATUS_DATE', 'SHIPNODE_KEY',
       'STATUS_DATE_COL']]

Consolidado_cruce =Consolidado_cruce[~Consolidado_cruce.SHIPNODE_KEY.isnull()]

inicio_servicio = Consolidado_cruce["iniciado"]
horas=[]
for element in inicio_servicio:
    horas.append(element.hour)
Consolidado_cruce["Hora_servicio"] = horas


inicio_servicio = Consolidado["iniciado"]
horas=[]
for element in inicio_servicio:
    horas.append(element.hour)
Consolidado["Hora_servicio"] = horas

fecha=date.today()

Consolidado_cruce["Diferencia_integracion"] = Consolidado_cruce["iniciado"] - Consolidado_cruce["STATUS_DATE_COL"]
diferencias = Consolidado_cruce["Diferencia_integracion"]
diferencias_min=[]

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

Consolidado_cruce["Packin_vs_inicio_3pl"] = diferencias_min

Consolidado_cruce=Consolidado_cruce[Consolidado_cruce["Dia"]==fecha.day]
Consolidado=Consolidado[Consolidado["Dia"]==fecha.day]

Consolidado_Sameday=pd.merge(Consolidado_cruce,resumen_total,left_on="numorden",right_on="ORDER_NO",how="left")


Consolidado_Sameday=Consolidado_Sameday[Consolidado_Sameday["LEVEL_OF_SERVICE"]=="SameDay"]

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/1149328331.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/1149328331.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/1149328331.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [80]:
df_asignado = Consolidado_Sameday[~Consolidado_Sameday.asignado.isnull()]
df_recogido = Consolidado_Sameday[(~Consolidado_Sameday.asignado.isnull()) & (~Consolidado_Sameday.llego_punto.isnull()) ]
df_salio_punto = Consolidado_Sameday[(~Consolidado_Sameday.llego_punto.isnull()) & (~Consolidado_Sameday.salio_punto.isnull()) ]
df_ruta = Consolidado_Sameday[(~Consolidado_Sameday.salio_punto.isnull()) & (~Consolidado_Sameday.llego_cliente.isnull()) ]
df_tiempo_um = Consolidado_Sameday[~Consolidado_Sameday.llego_cliente.isnull()]


df_asignado["asignado"] = pd.to_datetime(df_asignado["asignado"], format = "%Y-%m-%d %H:%M:%S")
df_asignado["Asignacion"] = df_asignado["asignado"] - df_asignado["iniciado"] 
diferencias = df_asignado["Asignacion"]
diferencias_min=[]                                               ###Calculos tiempos asignacion, recogida, punto y ruta en minutos

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

df_asignado["Asignacion_(min)"] = diferencias_min 

df_asignado.loc[df_asignado["Asignacion_(min)"]<0,"Asignacion_(min)"] = 0

df_recogido["llego_punto"] = pd.to_datetime(df_recogido["llego_punto"], format = "%Y-%m-%d %H:%M:%S")
df_recogido["asignado"]  = pd.to_datetime(df_recogido["asignado"] , format = "%Y-%m-%d %H:%M:%S")


df_recogido["Recogida"] = df_recogido["llego_punto"] -  df_recogido["asignado"] 
diferencias = df_recogido["Recogida"]
diferencias_min=[]                                               ###Calculos tiempos asignacion, recogida, punto y ruta en minutos

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

df_recogido["Recogida_(min)"] = diferencias_min

df_recogido.loc[df_recogido["Recogida_(min)"]<0,"Recogida_(min)"] = 0 

df_salio_punto["salio_punto"]  = pd.to_datetime(df_salio_punto["salio_punto"] , format = "%Y-%m-%d %H:%M:%S")
df_salio_punto["llego_punto"]  = pd.to_datetime(df_salio_punto["llego_punto"] , format = "%Y-%m-%d %H:%M:%S")


df_salio_punto["TiempoPunto"] = df_salio_punto["salio_punto"] -  df_salio_punto["llego_punto"] 
diferencias = df_salio_punto["TiempoPunto"]
diferencias_min=[]                                               ###Calculos tiempos asignacion, recogida, punto y ruta en minutos

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

df_salio_punto["TiempoPunto_(min)"] = diferencias_min

df_salio_punto.loc[df_salio_punto["TiempoPunto_(min)"]<0,"TiempoPunto_(min)"] = 0

df_ruta["llego_cliente"] = pd.to_datetime(df_ruta["llego_cliente"] , format = "%Y-%m-%d %H:%M:%S")
df_ruta["salio_punto"]  = pd.to_datetime(df_ruta["salio_punto"] , format = "%Y-%m-%d %H:%M:%S")

df_ruta["TiempoRuta"] = df_ruta["llego_cliente"] -  df_ruta["salio_punto"] 
diferencias = df_ruta["TiempoRuta"]
diferencias_min=[]                                               ###Calculos tiempos asignacion, recogida, punto y ruta en minutos

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

df_ruta["TiempoRuta_(min)"] = diferencias_min

df_ruta.loc[df_ruta["TiempoRuta_(min)"]<0,"TiempoRuta_(min)"] = 0

df_tiempo_um["llego_cliente"]  = pd.to_datetime( df_tiempo_um["llego_cliente"], format = "%Y-%m-%d %H:%M:%S")
df_tiempo_um["asignado"]  = pd.to_datetime(df_tiempo_um["asignado"], format = "%Y-%m-%d %H:%M:%S")


df_tiempo_um["TiempoUM"] = df_tiempo_um["llego_cliente"] -  df_tiempo_um["iniciado"] 
diferencias = df_tiempo_um["TiempoUM"]
diferencias_min=[]                                               ###Calculos tiempos asignacion, recogida, punto y ruta en minutos

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)

df_tiempo_um["TiempoUM_(min)"] = diferencias_min

df_tiempo_um.loc[df_tiempo_um["TiempoUM_(min)"]<0,"TiempoUM_(min)"] = 0


df_tiempo_um["TiempoTotal"] = df_tiempo_um["llego_cliente"] -  df_tiempo_um["ORDER_DATE_COL"]
diferencias = df_tiempo_um["TiempoTotal"] 
diferencias_min =[]

for element in diferencias:
    diferencias_min.append(element.total_seconds()/60)
    

df_tiempo_um["TiempoTotal_(min)"] = diferencias_min


####Cumplimientos 

df_asignado["CumpleAsignacion"] = "No Cumple" 
df_recogido["CumpleRecogida"] = "No Cumple" 
df_salio_punto["CumplePunto"] = "No Cumple" 
df_ruta["CumpleRuta"] = "No Cumple"
df_tiempo_um["CumpleTiempoTotal"] = "No Cumple"
df_tiempo_um["CumpleTiempoCreacion"] = "No Cumple"
Consolidado_Sameday["DemoraIntegracion"] = "Sin Novedad"

df_asignado.loc[df_asignado["Asignacion_(min)"]<=5,"CumpleAsignacion"]="Cumple"
df_recogido.loc[df_recogido["Recogida_(min)"]<=7,"CumpleRecogida"]="Cumple"
df_salio_punto.loc[df_salio_punto["TiempoPunto_(min)"]<=7,"CumplePunto"] = "Cumple"
df_ruta.loc[df_ruta["TiempoRuta_(min)"]<=18,"CumpleRuta"] = "Cumple"
df_tiempo_um.loc[df_tiempo_um["TiempoUM_(min)"]<=40,"CumpleTiempoTotal"] = "Cumple"
df_tiempo_um.loc[df_tiempo_um["TiempoTotal_(min)"]<=60,"CumpleTiempoCreacion"] = "Cumple"
Consolidado_Sameday.loc[Consolidado_Sameday["Packin_vs_inicio_3pl"]>7,"DemoraIntegracion"] = "Con Novedad"

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/3137153629.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/3137153629.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/3137153629.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [81]:
Consolidado_Sameday = Consolidado_Sameday[['numorden', 'iniciado', 'asignado', 'llego_punto', 'salio_punto',
       'llego_cliente', 'finalizado', 'distancia_km', 'Finalizado Fallido',
       'Tipo Fallido', 'Valor Servicio', 'Proveedor', 'Estado UM', 'Mes',
       'Dia', 'STATUS_DATE', 'SHIPNODE_KEY', 'STATUS_DATE_COL',
       'Hora_servicio', 'Diferencia_integracion', 'Packin_vs_inicio_3pl',
       'ORDER_NO', 'LEVEL_OF_SERVICE','EXTN_ORG_REQ_SHIP_DATE',"ORDER_DATE_COL","Ciudad", 'DemoraIntegracion']]

Consolidado_Sameday.columns = ['numorden', 'iniciado', 'asignado', 'llego_punto', 'salio_punto',
       'llego_cliente', 'finalizado', 'distancia_km', 'Finalizado_Fallido',
       'Tipo_Fallido', 'Valor_Servicio', 'Proveedor', 'Estado_UM', 'Mes',
       'Dia', 'Packing Complete', 'SHIPNODE_KEY', 'Packing_Complete_COL',
       'Hora_servicio', 'Diferencia_integracion', 'Packin_vs_inicio_3pl',
       'ORDER_NO', 'LEVEL_OF_SERVICE','EXTN_ORG_REQ_SHIP_DATE',"ORDER_DATE_COL","Ciudad", 'DemoraIntegracion']

In [ ]:
pivot_asignacion = pd.pivot_table(df_asignado,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],values=["Asignacion_(min)"],aggfunc=np.mean)
pivot_recogida = pd.pivot_table(df_recogido,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],values=["Recogida_(min)"],aggfunc=np.mean)
pivot_tiempo_punto = pd.pivot_table(df_salio_punto,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],values=["TiempoPunto_(min)"],aggfunc = np.mean)
pivot_tiempo_ruta = pd.pivot_table(df_ruta, index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],values=["TiempoRuta_(min)"],aggfunc = np.mean)
pivot_tiempo_total = pd.pivot_table(df_tiempo_um, index = ["Proveedor","SHIPNODE_KEY","Hora_servicio"], values= ["TiempoUM_(min)"],aggfunc = np.mean)
pivot_dif_packin_iniciado = pd.pivot_table(Consolidado_Sameday,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"] , values=["Packin_vs_inicio_3pl"], aggfunc=np.mean)
pivot_tiempo_creacion = pd.pivot_table(df_tiempo_um, index = ["Proveedor","SHIPNODE_KEY","Hora_servicio"], values= ["TiempoTotal_(min)"],aggfunc = np.mean)


pivot_asignacion_p = pd.pivot_table(df_asignado,index=["Proveedor","Hora_servicio"],values=["Asignacion_(min)"],aggfunc=np.mean)
pivot_recogida_p = pd.pivot_table(df_recogido,index=["Proveedor","Hora_servicio"],values=["Recogida_(min)"],aggfunc=np.mean)
pivot_tiempo_punto_p = pd.pivot_table(df_salio_punto,index=["Proveedor","Hora_servicio"],values=["TiempoPunto_(min)"],aggfunc = np.mean)
pivot_tiempo_ruta_p = pd.pivot_table(df_ruta, index=["Proveedor","Hora_servicio"],values=["TiempoRuta_(min)"],aggfunc = np.mean)
pivot_tiempo_total_p = pd.pivot_table(df_tiempo_um, index = ["Proveedor","Hora_servicio"], values= ["TiempoUM_(min)"],aggfunc = np.mean)
pivot_dif_packin_iniciado_p = pd.pivot_table(Consolidado_Sameday,index=["Proveedor","Hora_servicio"] , values=["Packin_vs_inicio_3pl"], aggfunc=np.mean)
pivot_tiempo_creacion_p = pd.pivot_table(df_tiempo_um, index = ["Proveedor","Hora_servicio"], values= ["TiempoTotal_(min)"],aggfunc = np.mean)


pivot_cumple_asignacion = pd.pivot_table(df_asignado,index = ["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns=["CumpleAsignacion"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_cumple_recogida = pd.pivot_table(df_recogido,index = ["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns=["CumpleRecogida"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_cumple_tiempo_punto = pd.pivot_table(df_salio_punto, index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns= ["CumplePunto"],values=["numorden"],aggfunc ="count",fill_value=0)
pivot_cumple_ruta = pd.pivot_table(df_ruta,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns=["CumpleRuta"],values=["numorden"],aggfunc="count",fill_value=0)
pivot_cumple_total = pd.pivot_table(df_tiempo_um,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns=["CumpleTiempoTotal"],values = ["numorden"],aggfunc = "count",fill_value=0)
pivot_participacion_ = pd.pivot_table(Consolidado_Sameday,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns = ["DemoraIntegracion"],values = ["numorden"],aggfunc = "count",fill_value = 0)
pivot_cumple_tcreacion = pd.pivot_table(df_tiempo_um,index=["Proveedor","SHIPNODE_KEY","Hora_servicio"],columns=["CumpleTiempoCreacion"],values = ["numorden"],aggfunc = "count",fill_value=0)

pivot_cumple_asignacion_p = pd.pivot_table(df_asignado,index = ["Proveedor","Hora_servicio"],columns=["CumpleAsignacion"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_cumple_recogida_p = pd.pivot_table(df_recogido,index = ["Proveedor","Hora_servicio"],columns=["CumpleRecogida"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_cumple_tiempo_punto_p = pd.pivot_table(df_salio_punto, index=["Proveedor","Hora_servicio"],columns= ["CumplePunto"],values=["numorden"],aggfunc ="count",fill_value=0)
pivot_cumple_ruta_p = pd.pivot_table(df_ruta,index=["Proveedor","Hora_servicio"],columns=["CumpleRuta"],values=["numorden"],aggfunc="count",fill_value=0)
pivot_cumple_total_p = pd.pivot_table(df_tiempo_um,index=["Proveedor","Hora_servicio"],columns=["CumpleTiempoTotal"],values = ["numorden"],aggfunc = "count",fill_value=0)
pivot_participacion_p = pd.pivot_table(Consolidado_Sameday,index=["Proveedor","Hora_servicio"],columns = ["DemoraIntegracion"],values = ["numorden"],aggfunc = "count",fill_value = 0)
pivot_cumple_tcreacion_p = pd.pivot_table(df_tiempo_um,index=["Proveedor","Hora_servicio"],columns=["CumpleTiempoCreacion"],values = ["numorden"],aggfunc = "count",fill_value=0)

pivot_cumple_asignacion[("numorden","Cumplimiento")] = pivot_cumple_asignacion[("numorden","Cumple")]/(pivot_cumple_asignacion[("numorden","Cumple")] + pivot_cumple_asignacion[("numorden","No Cumple")])
pivot_cumple_recogida[("numorden","Cumplimiento")] = pivot_cumple_recogida[("numorden","Cumple")]/(pivot_cumple_recogida[("numorden","Cumple")] + pivot_cumple_recogida[("numorden","No Cumple")])
pivot_cumple_tiempo_punto[("numorden","Cumplimiento")] = pivot_cumple_tiempo_punto[("numorden","Cumple")] / (pivot_cumple_tiempo_punto[("numorden","Cumple")] + pivot_cumple_tiempo_punto[("numorden","No Cumple")])
pivot_cumple_ruta[("numorden","Cumplimiento")] = pivot_cumple_ruta[("numorden","Cumple")] / (pivot_cumple_ruta[("numorden","Cumple")] + pivot_cumple_ruta[("numorden","No Cumple")])
pivot_cumple_total[("numorden","Cumplimiento")]  = pivot_cumple_total[("numorden","Cumple")] / (pivot_cumple_total[("numorden","Cumple")] + pivot_cumple_total[("numorden","No Cumple")])
pivot_participacion_[("numorden","Participacion")] = pivot_participacion_[("numorden","Con Novedad")] / (pivot_participacion_[("numorden","Con Novedad")] + pivot_participacion_[("numorden","Sin Novedad")])
pivot_cumple_tcreacion[("numorden","Cumplimiento")]  = pivot_cumple_tcreacion[("numorden","Cumple")] / (pivot_cumple_tcreacion[("numorden","Cumple")] + pivot_cumple_tcreacion[("numorden","No Cumple")])

pivot_cumple_asignacion_p[("numorden","Cumplimiento")] = pivot_cumple_asignacion_p[("numorden","Cumple")]/(pivot_cumple_asignacion_p[("numorden","Cumple")] + pivot_cumple_asignacion_p[("numorden","No Cumple")])
pivot_cumple_recogida_p[("numorden","Cumplimiento")] = pivot_cumple_recogida_p[("numorden","Cumple")]/(pivot_cumple_recogida_p[("numorden","Cumple")] + pivot_cumple_recogida_p[("numorden","No Cumple")])
pivot_cumple_tiempo_punto_p[("numorden","Cumplimiento")] = pivot_cumple_tiempo_punto_p[("numorden","Cumple")] / (pivot_cumple_tiempo_punto_p[("numorden","Cumple")] + pivot_cumple_tiempo_punto_p[("numorden","No Cumple")])
pivot_cumple_ruta_p[("numorden","Cumplimiento")] = pivot_cumple_ruta_p[("numorden","Cumple")] / (pivot_cumple_ruta_p[("numorden","Cumple")] + pivot_cumple_ruta_p[("numorden","No Cumple")])
pivot_cumple_total_p[("numorden","Cumplimiento")]  = pivot_cumple_total_p[("numorden","Cumple")] / (pivot_cumple_total_p[("numorden","Cumple")] + pivot_cumple_total_p[("numorden","No Cumple")])
pivot_participacion_p[("numorden","Participacion")] = pivot_participacion_p[("numorden","Con Novedad")] / (pivot_participacion_p[("numorden","Con Novedad")] + pivot_participacion_p[("numorden","Sin Novedad")])
pivot_cumple_tcreacion_p[("numorden","Cumplimiento")]  = pivot_cumple_tcreacion_p[("numorden","Cumple")] / (pivot_cumple_tcreacion_p[("numorden","Cumple")] + pivot_cumple_tcreacion_p[("numorden","No Cumple")])

Top_packin_vs_iniciado =Consolidado_Sameday[Consolidado_Sameday["Packin_vs_inicio_3pl"]>10]
Top_packin_vs_iniciado = Top_packin_vs_iniciado.sort_values(by = ["Packin_vs_inicio_3pl"],ascending = False )
Top_packin_vs_iniciado["Packin_vs_inicio_3pl"] = np.round(Top_packin_vs_iniciado["Packin_vs_inicio_3pl"],decimals = 0)
Top_packin_vs_iniciado.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Top_packing_vs_iniciado.xlsx")
Ordenes_sin_3pl.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Posibles_Errores_en_Hub.xlsx")
Consolidado.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Consolidado.xlsx")


pivot_asignacion = pivot_asignacion.reset_index()
pivot_asignacion_p = pivot_asignacion_p.reset_index()
pivot_recogida = pivot_recogida.reset_index()
pivot_recogida_p = pivot_recogida_p.reset_index()
pivot_tiempo_punto  = pivot_tiempo_punto.reset_index()
pivot_tiempo_punto_p  = pivot_tiempo_punto_p.reset_index()
pivot_tiempo_ruta = pivot_tiempo_ruta.reset_index()
pivot_tiempo_ruta_p = pivot_tiempo_ruta_p.reset_index()
pivot_tiempo_total = pivot_tiempo_total.reset_index()
pivot_tiempo_total_p = pivot_tiempo_total_p.reset_index()
pivot_dif_packin_iniciado = pivot_dif_packin_iniciado.reset_index()
pivot_dif_packin_iniciado_p = pivot_dif_packin_iniciado_p.reset_index()

pivot_cumple_asignacion = pivot_cumple_asignacion.reset_index()
pivot_cumple_recogida = pivot_cumple_recogida.reset_index()
pivot_cumple_tiempo_punto = pivot_cumple_tiempo_punto.reset_index()
pivot_cumple_ruta = pivot_cumple_ruta.reset_index()
pivot_cumple_total = pivot_cumple_total.reset_index()
pivot_participacion_ = pivot_participacion_.reset_index()
pivot_cumple_asignacion_p = pivot_cumple_asignacion_p.reset_index()
pivot_cumple_recogida_p = pivot_cumple_recogida_p.reset_index()
pivot_cumple_tiempo_punto_p = pivot_cumple_tiempo_punto_p.reset_index()
pivot_cumple_ruta_p = pivot_cumple_ruta_p.reset_index()
pivot_cumple_total_p = pivot_cumple_total_p.reset_index()
pivot_participacion_p = pivot_participacion_p.reset_index()

pivot_asignacion.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Asignacion (min)"]
pivot_asignacion_p.columns = ["Proveedor","Hora_Servicio","Tiempo Asignacion (min)"]
pivot_asignacion["Tiempo Asignacion (min)"] = np.round(pivot_asignacion["Tiempo Asignacion (min)"],decimals=0 )
pivot_asignacion_p["Tiempo Asignacion (min)"] = np.round(pivot_asignacion_p["Tiempo Asignacion (min)"],decimals=0 )

pivot_recogida.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Recogida (min)"]
pivot_recogida_p.columns = ["Proveedor","Hora_Servicio","Tiempo Recogida (min)"]
pivot_recogida["Tiempo Recogida (min)"] = np.round(pivot_recogida["Tiempo Recogida (min)"],decimals =0)
pivot_recogida_p["Tiempo Recogida (min)"] = np.round(pivot_recogida_p["Tiempo Recogida (min)"],decimals = 0)

pivot_tiempo_punto.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Punto (min)"]
pivot_tiempo_punto_p.columns = ["Proveedor","Hora_Servicio","Tiempo Punto (min)"]
pivot_tiempo_punto["Tiempo Punto (min)"] = np.round(pivot_tiempo_punto["Tiempo Punto (min)"],decimals = 0)
pivot_tiempo_punto_p["Tiempo Punto (min)"] = np.round(pivot_tiempo_punto_p["Tiempo Punto (min)"],decimals =0)

pivot_tiempo_ruta.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Ruta (min)"]
pivot_tiempo_ruta_p.columns = ["Proveedor","Hora_Servicio","Tiempo Ruta (min)"]
pivot_tiempo_ruta["Tiempo Ruta (min)"] = np.round(pivot_tiempo_ruta["Tiempo Ruta (min)"],decimals = 0)
pivot_tiempo_ruta_p["Tiempo Ruta (min)"] = np.round(pivot_tiempo_ruta_p["Tiempo Ruta (min)"],decimals= 0)

pivot_tiempo_total.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Total (min)"]
pivot_tiempo_total_p.columns = ["Proveedor","Hora_Servicio","Tiempo Total (min)"]
pivot_tiempo_total["Tiempo Total (min)"] = np.round(pivot_tiempo_total["Tiempo Total (min)"],decimals = 0)
pivot_tiempo_total_p["Tiempo Total (min)"] = np.round(pivot_tiempo_total_p["Tiempo Total (min)"],decimals= 0)

pivot_dif_packin_iniciado.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Tiempo Integracion (min)"]
pivot_dif_packin_iniciado_p.columns = ["Proveedor","Hora_Servicio","Tiempo Integracion (min)"]
pivot_dif_packin_iniciado["Tiempo Integracion (min)"] = np.round(pivot_dif_packin_iniciado["Tiempo Integracion (min)"],decimals = 0)
pivot_dif_packin_iniciado_p["Tiempo Integracion (min)"] = np.round(pivot_dif_packin_iniciado_p["Tiempo Integracion (min)"],decimals = 0)

pivot_cumple_asignacion.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_asignacion_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_asignacion["Cumplimiento"] = np.round(pivot_cumple_asignacion["Cumplimiento"]*100,decimals = 1 )
pivot_cumple_asignacion_p["Cumplimiento"] = np.round(pivot_cumple_asignacion_p["Cumplimiento"]*100,decimals = 1)

pivot_cumple_recogida.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_recogida_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_recogida["Cumplimiento"] = np.round(pivot_cumple_recogida["Cumplimiento"]*100,decimals = 1)
pivot_cumple_recogida_p["Cumplimiento"] = np.round(pivot_cumple_recogida_p["Cumplimiento"]*100,decimals = 1)

pivot_cumple_tiempo_punto.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_tiempo_punto_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_tiempo_punto["Cumplimiento"] = np.round(pivot_cumple_tiempo_punto["Cumplimiento"]*100,decimals = 1)
pivot_cumple_tiempo_punto_p["Cumplimiento"] = np.round(pivot_cumple_tiempo_punto_p["Cumplimiento"]*100,decimals =1)

pivot_cumple_ruta.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_ruta_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_ruta["Cumplimiento"] = np.round(pivot_cumple_ruta["Cumplimiento"]*100,decimals = 1)
pivot_cumple_ruta_p["Cumplimiento"] = np.round(pivot_cumple_ruta_p["Cumplimiento"]*100,decimals= 1)

pivot_cumple_total.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_total_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Cumplimiento"]
pivot_cumple_total["Cumplimiento"] = np.round(pivot_cumple_total["Cumplimiento"]*100, decimals =1)
pivot_cumple_total_p["Cumplimiento"] = np.round(pivot_cumple_total_p["Cumplimiento"]*100,decimals = 1)

pivot_participacion_.columns = ["Proveedor","SHIPNODE_KEY","Hora_Servicio","Cumple","No Cumple","Participacion"]
pivot_participacion_p.columns = ["Proveedor","Hora_Servicio","Cumple","No Cumple","Participacion"]
pivot_participacion_["Participacion"] = np.round(pivot_participacion_["Participacion"]*100,decimals = 1)
pivot_participacion_p["Participacion"] = np.round(pivot_participacion_p["Participacion"]*100,decimals = 1)


pivot_asignacion_punto = pd.pivot_table(df_asignado,index = ["Proveedor","SHIPNODE_KEY"],columns=["CumpleAsignacion"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_asignacion_punto[("numorden","Cumplimiento")] = pivot_asignacion_punto[("numorden","Cumple")]/(pivot_asignacion_punto[("numorden","Cumple")] + pivot_asignacion_punto[("numorden","No Cumple")])
pivot_asignacion_punto = pivot_asignacion_punto.reset_index()

pivot_recogida_punto = pd.pivot_table(df_recogido,index = ["Proveedor","SHIPNODE_KEY"],columns=["CumpleRecogida"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_recogida_punto[("numorden","Cumplimiento")] = pivot_recogida_punto[("numorden","Cumple")]/(pivot_recogida_punto[("numorden","Cumple")] + pivot_recogida_punto[("numorden","No Cumple")])
pivot_recogida_punto= pivot_recogida_punto.reset_index()


pivot_punto_punto = pd.pivot_table(df_salio_punto,index = ["Proveedor","SHIPNODE_KEY"],columns=["CumplePunto"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_punto_punto[("numorden","Cumplimiento")] = pivot_punto_punto[("numorden","Cumple")]/(pivot_punto_punto[("numorden","Cumple")] + pivot_punto_punto[("numorden","No Cumple")])
pivot_punto_punto= pivot_punto_punto.reset_index()

pivot_ruta_punto = pd.pivot_table(df_ruta,index = ["Proveedor","SHIPNODE_KEY"],columns=["CumpleRuta"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_ruta_punto[("numorden","Cumplimiento")] = pivot_ruta_punto[("numorden","Cumple")]/(pivot_ruta_punto[("numorden","Cumple")] + pivot_ruta_punto[("numorden","No Cumple")])
pivot_ruta_punto= pivot_ruta_punto.reset_index()

pivot_total_punto = pd.pivot_table(df_tiempo_um,index = ["Proveedor","SHIPNODE_KEY"],columns=["CumpleTiempoTotal"],values=["numorden"],aggfunc="count",fill_value=0 )
pivot_total_punto[("numorden","Cumplimiento")] = pivot_total_punto[("numorden","Cumple")]/(pivot_total_punto[("numorden","Cumple")] + pivot_total_punto[("numorden","No Cumple")])
pivot_total_punto= pivot_total_punto.reset_index()


pivot_asignacion_punto.columns = ["Proveedor","Sucursal","Cumple","No Cumple","Cumplimiento"]
pivot_recogida_punto.columns = ["Proveedor","Sucursal","Cumple","No Cumple","Cumplimiento"]
pivot_punto_punto.columns = ["Proveedor","Sucursal","Cumple","No Cumple","Cumplimiento"]
pivot_ruta_punto.columns = ["Proveedor","Sucursal","Cumple","No Cumple","Cumplimiento"]
pivot_total_punto.columns = ["Proveedor","Sucursal","Cumple","No Cumple","Cumplimiento"]

pivot_asignacion_punto = pivot_asignacion_punto[["Proveedor","Sucursal","Cumplimiento"]]
pivot_recogida_punto = pivot_recogida_punto[["Proveedor","Sucursal","Cumplimiento"]]
pivot_punto_punto = pivot_punto_punto[["Proveedor","Sucursal","Cumplimiento"]]
pivot_ruta_punto = pivot_ruta_punto[["Proveedor","Sucursal","Cumplimiento"]]
pivot_total_punto = pivot_total_punto[["Proveedor","Sucursal","Cumplimiento"]]


pivot_asignacion_punto = pivot_asignacion_punto.sort_values(by = ["Cumplimiento"],ascending = True)
pivot_recogida_punto = pivot_recogida_punto.sort_values(by = ["Cumplimiento"],ascending = True)
pivot_punto_punto = pivot_punto_punto.sort_values(by = ["Cumplimiento"],ascending = True)
pivot_ruta_punto = pivot_ruta_punto.sort_values(by = ["Cumplimiento"],ascending = True)
pivot_total_punto = pivot_total_punto.sort_values(by = ["Cumplimiento"],ascending = True)

pivot_asignacion_punto["Cumplimiento"] = np.round(pivot_asignacion_punto["Cumplimiento"]*100,decimals = 1)
pivot_recogida_punto["Cumplimiento"] = np.round(pivot_recogida_punto["Cumplimiento"]*100,decimals= 1)
pivot_punto_punto["Cumplimiento"] = np.round(pivot_punto_punto["Cumplimiento"]*100,decimals =1)
pivot_ruta_punto["Cumplimiento"] = np.round(pivot_ruta_punto["Cumplimiento"]*100,decimals =1)
pivot_total_punto["Cumplimiento"] = np.round(pivot_total_punto["Cumplimiento"]*100,decimals = 1)


hora_actual  = datetime.datetime.now()

df_finalizados = Consolidado_Sameday[Consolidado_Sameday.Estado_UM.isin(["Finalizado"])]
df_sin_asignar = Consolidado_Sameday[Consolidado_Sameday.Estado_UM.isin(["Creado","En Espera","En espera"])]

horas_inicio = df_sin_asignar["iniciado"]
tiempo_calculo = []
for elemento in horas_inicio:
    dif = hora_actual - elemento
    dif = (dif.total_seconds()/60)
    tiempo_calculo.append(dif)
    
df_sin_asignar["TiempoLive"] = tiempo_calculo
df_sin_asignar = df_sin_asignar[["numorden","distancia_km","Proveedor","Ciudad","TiempoLive"]]
df_sin_asignar = df_sin_asignar.sort_values(by=["TiempoLive"],ascending = False)
df_sin_asignar["TiempoLive"] = np.round(df_sin_asignar["TiempoLive"],decimals =0)


df_sin_recoger =  Consolidado_Sameday[Consolidado_Sameday.Estado_UM.isin(["Asignado"])]
horas_asignado = df_sin_recoger["asignado"]
tiempo_calculo = []

for elemento in horas_asignado:
    dif = hora_actual - elemento
    dif = (dif.total_seconds()/60)
    tiempo_calculo.append(dif)

df_sin_recoger ["TiempoLive"] = tiempo_calculo
df_sin_recoger = df_sin_recoger[["numorden","distancia_km","Proveedor","Ciudad","TiempoLive"]]
df_sin_recoger  = df_sin_recoger.sort_values(by=["TiempoLive"],ascending = False)
df_sin_recoger["TiempoLive"] = np.round(df_sin_recoger["TiempoLive"],decimals =0)

df_en_proceso = Consolidado_Sameday[Consolidado_Sameday.Estado_UM.isin(["En Proceso","En Transito", "En transito"])]

horas_llegada_punto = df_en_proceso["llego_punto"]
tiempo_calculo = []

for elemento in horas_llegada_punto:
    dif = hora_actual - elemento
    dif = dif.total_seconds()/60
    tiempo_calculo.append(dif)

df_en_proceso["TiempoLive"] = tiempo_calculo

df_en_proceso=df_en_proceso[["numorden","distancia_km","Proveedor","Ciudad","TiempoLive"]]
df_en_proceso = df_en_proceso.sort_values(by=["TiempoLive"],ascending = False)
df_en_proceso["TiempoLive"] = np.round(df_en_proceso["TiempoLive"],decimals= 0)


df_sin_asignar = df_sin_asignar[df_sin_asignar["TiempoLive"]>=10]
df_sin_recoger  = df_sin_recoger[df_sin_recoger["TiempoLive"]>=10]
df_en_proceso = df_en_proceso[df_en_proceso["TiempoLive"]>=30]

sin_asignarXciudad = pd.pivot_table(df_sin_asignar,index = ["Ciudad"], values = ["numorden"],aggfunc = "count")
sin_recogerXciudad = pd.pivot_table(df_sin_recoger,index = ["Ciudad"], values = ["numorden"],aggfunc = "count")
sin_cerrarXciudad = pd.pivot_table(df_en_proceso,index = ["Ciudad"], values = ["numorden"],aggfunc = "count")
finalizados = pd.pivot_table(df_finalizados,index = ["Ciudad"] , values = ["numorden"], aggfunc = "count")
                                                                         

tiempos_ciudad = pd.pivot_table(df_tiempo_um, index = ["Ciudad"], values= ["TiempoUM_(min)","TiempoTotal_(min)"],aggfunc = np.mean)
tiempos_ciudad  = tiempos_ciudad.reset_index()
tiempos_ciudad.columns = ["Ciudad","Tiempo desde creación (min)","Tiempo UM"]
tiempos_ciudad["Tiempo desde creación (min)"] = np.round(tiempos_ciudad["Tiempo desde creación (min)"],decimals = 0)
tiempos_ciudad["Tiempo UM"] = np.round(tiempos_ciudad["Tiempo UM"],decimals = 0)


cumple_tiempos_ciudad = pd.pivot_table(df_tiempo_um,index=["Ciudad"],columns=["CumpleTiempoTotal","CumpleTiempoCreacion"],values = ["numorden"],aggfunc = "count",fill_value=0)
cumple_tiempos_ciudad.reset_index()
cumple_tiempos_ciudad.columns =["CumpleUM-CumpleTotal","CumpleUM-NoCumpleTotal","NoCumpleUM-CumpleTotal","NoCumpleUM-NoCumpleTotal"]
cumple_tiempos_ciudad["CumplimientoUM"] = (cumple_tiempos_ciudad["CumpleUM-CumpleTotal"] + cumple_tiempos_ciudad["CumpleUM-NoCumpleTotal"])  /( cumple_tiempos_ciudad["NoCumpleUM-CumpleTotal"] +  cumple_tiempos_ciudad["NoCumpleUM-NoCumpleTotal"] + cumple_tiempos_ciudad["CumpleUM-CumpleTotal"] + cumple_tiempos_ciudad["CumpleUM-NoCumpleTotal"])
cumple_tiempos_ciudad["CumplimientoTiempoTotal"] = (cumple_tiempos_ciudad["CumpleUM-CumpleTotal"] + cumple_tiempos_ciudad["NoCumpleUM-CumpleTotal"]) /( cumple_tiempos_ciudad["NoCumpleUM-CumpleTotal"] +  cumple_tiempos_ciudad["NoCumpleUM-NoCumpleTotal"] + cumple_tiempos_ciudad["CumpleUM-CumpleTotal"] + cumple_tiempos_ciudad["CumpleUM-NoCumpleTotal"])
cumple_tiempos_ciudad = cumple_tiempos_ciudad.reset_index()
cumple_tiempos_ciudad.columns = ["Ciudad","CumpleUM-CumpleTotal","CumpleUM-NoCumpleTotal","NoCumpleUM-CumpleTotal","NoCumpleUM-NoCumpleTotal","CumplimientoUM","CumplimientoTiempoTotal"]
cumple_tiempos_ciudad["CumplimientoUM"] = np.round(cumple_tiempos_ciudad["CumplimientoUM"]*100,decimals = 1)
cumple_tiempos_ciudad["CumplimientoTiempoTotal"] = np.round(cumple_tiempos_ciudad["CumplimientoTiempoTotal"]*100,decimals = 1)


sin_asignarXciudad=sin_asignarXciudad.reset_index()
sin_recogerXciudad = sin_recogerXciudad.reset_index()
sin_cerrarXciudad = sin_cerrarXciudad.reset_index()
finalizados = finalizados.reset_index()

sin_asignarXciudad.columns =["Ciudad","Ordenes sin asignar"]
sin_recogerXciudad.columns = ["Ciudad","Ordenes sin recoger"]
sin_cerrarXciudad.columns = ["Ciudad","Ordenes sin cerrar"]
finalizados.columns = ["Ciudad","Ordenes finalizadas"]


df_sin_asignar.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Criticas_Asignacion.xlsx")
df_sin_recoger.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Criticas_Recogida.xlsx")
df_en_proceso.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\Criticas_Cierre.xlsx")

locales=Consolidado_Sameday["SHIPNODE_KEY"]
locales = list(locales.unique())
locales.append("Todos")
locales.sort()

fig_ordenes_sin3pl = go.Table(header = dict(values=["Orden","Sucursal","Fecha Packing"]),
                             cells = dict(values =[Ordenes_sin_3pl["ORDER_NO"], Ordenes_sin_3pl["SHIPNODE_KEY"],Ordenes_sin_3pl["STATUS_DATE_COL"]]))



app = Dash(__name__)

app.layout = html.Div([
    
    html.Div([
            html.H1("Gestión Última Milla -Diario-")
        ]),
    
    
    html.Div([
        html.Div([
            dcc.Graph(id ="sin_asig_ciudad")],style ={'float':'left','width':'24%'}),
        
        html.Div([
            dcc.Graph(id="sin_rec_ciudad")],style ={'float':'left','width':'24%'}),
        
        html.Div([
            dcc.Graph(id="sin_cerrar_ciudad")],style ={'float':'left','width':'24%'}),
        html.Div([
            dcc.Graph(id = "finalizados")],style ={'float':'left','width':'24%'})
        
    ]),
    
    html.Div([
    
        html.Div([
            dcc.Graph(id="tiempos_promedio_ciudad")],style ={'float':'left','width':'49%'}),
        
        html.Div([
            dcc.Graph(id = "cumplimientos_tiempos_ciudad")], style ={'float':'left','width':'49%'})
        
        
    
    ]),
    
    
    html.Div([
        
        
        html.Div([
            
            html.P('Seleccionar proveedor',style={'color':'black','margin-top':'2px'}),
            dcc.Dropdown(["MU","Picap","Quick","Alianza"],"MU",id="dropdown_prov")
        ],style = {'width':'100%','float':'left'})
        
        
        
        
    ]),

    
    html.Div([
        
        html.Div([
            dcc.Graph(id = "criticas_asig")],style = {'float':'left','width':'33%','padding':'1px'}),
        
        html.Div([
            dcc.Graph(id ="criticas_rec")],style ={'float':'left','width':'33%','padding':'1px'}),
        
        
        html.Div([
            dcc.Graph(id = "criticas_cierre")],style = {'float':'left','width':'33%','padding':'1px'})
        
    ],style ={'padding':'1px'}),
    
    
    html.Div([
        
        
       html.Div([
           
           dcc.Graph(id="demora_int")
           
           
       ],style = {'float':'left','width':'90%'}),
        
        html.Div([
            dcc.Graph(id="sin3pl", figure ={'data':[fig_ordenes_sin3pl],'layout':go.Layout(title = "Ordenes con posible error en Hub")})
        ],style ={'float':'left','width':'90%'})
        
        
        
    ]),
    
    html.Div([
        
        html.Div([
            dcc.Graph(id = "puntos_asignacion")], style = {'float':'left','width':'20%','padding':'1px'}),
        
        html.Div([
            dcc.Graph(id= "puntos_recogida")], style = {'float':'left','width':'20%','padding':'1px'}),
        
        html.Div([
            dcc.Graph(id = "puntos_punto")], style = { 'float':'left','width':'18%','padding':'1px'}),
        
        html.Div([
            dcc.Graph(id= "puntos_ruta")], style = {'float':'left','width':'20%','padding':'1px'}),
        
        html.Div([
            dcc.Graph(id="puntos_total")], style = {'float':'left','width':'20%','padding':'1px'})
    
        
    ]),
    
    html.Div([
        
        html.Div([
            
            html.P('Seleccionar punto de venta',style={'color':'black','margin-top':'2px'}),
            dcc.Dropdown(locales,"Todos",id ="dropdown_pdv")
        ],style ={'width':'100%','float':'left'})
        
        
      
        
    ]),
    
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoAsignacion")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="CumplimientoAsignacion")],style={'width':'50%','float':'right'})

     ]),
    
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoRecogida")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="CumplimientoRecogida")],style={'width':'50%','float':'right'})
        
        
        
        
    ]),
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoPunto")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="CumplimientoTiempoPunto")],style={'width':'50%','float':'right'})
        
    ]),
    
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoRuta")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="CumplimientoTiempoRuta")],style={'width':'50%','float':'right'})
        
    ]),
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoTotal")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="CumplimientoTiempoTotal")],style={'width':'50%','float':'right'})
        
    ]),
    
    html.Div([
        
        html.Div([
            dcc.Graph(id="TiempoIntegracion")],style ={'width':'50%','float':'left'}),
        
        html.Div([
            dcc.Graph(id="Participacion")],style={'width':'50%','float':'right'})
        
    ])
])


@app.callback(
    Output("sin_asig_ciudad","figure"),
    Input("dropdown_prov","value"))
def update_sin_asignar_ciudad(value):
    
    filtro = sin_asignarXciudad
    
    fig_sin_asignar_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Ordenes sin asignar"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["Ordenes sin asignar"]]))])
    
    fig_sin_asignar_ciudad.update_layout(title ="Ordenes sin asignar")
    
    return fig_sin_asignar_ciudad

@app.callback(
    Output("sin_rec_ciudad","figure"),
    Input("dropdown_prov","value"))
def update_sin_recoger_ciudad(value):
    
    filtro = sin_recogerXciudad
    
    fig_sin_recoger_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Ordenes sin recoger"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["Ordenes sin recoger"]]))])

    
    fig_sin_recoger_ciudad.update_layout(title ="Ordenes sin recoger")
    
    return fig_sin_recoger_ciudad


@app.callback(
    Output("sin_cerrar_ciudad","figure"),
    Input("dropdown_prov","value"))
def update_sin_cerrar_ciudad(value):
    
    filtro = sin_cerrarXciudad
    
    fig_sin_cerrar_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Ordenes sin cerrar"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["Ordenes sin cerrar"]]))])

    fig_sin_cerrar_ciudad.update_layout(title ="Ordenes sin cerrar")
    
    return fig_sin_cerrar_ciudad

@app.callback(
    Output("finalizados","figure"),
    Input("dropdown_prov","value"))
def update_finalizados(value):
    
    filtro  = finalizados
    
    fig_finalizados_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Ordenes finalizadas"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["Ordenes finalizadas"]]))])

    
    fig_finalizados_ciudad.update_layout(title = "Ordenes finalizadas")
    
    return fig_finalizados_ciudad

@app.callback(
    Output("tiempos_promedio_ciudad","figure"),
    Input("dropdown_prov","value"))
def update_tiempos_ciudad(value):
    
    filtro = tiempos_ciudad

    fig_tiempos_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Tiempos UM","Tiempos desde creación"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["Tiempo UM"], filtro["Tiempo desde creación (min)"]]))])

    
    fig_tiempos_ciudad.update_layout(title = "Tiempos promedio por ciudad")
    
    return fig_tiempos_ciudad

@app.callback(
    Output("cumplimientos_tiempos_ciudad","figure"),
    Input("dropdown_prov","value"))
def update_cumple_tiempos_ciudad(value):
    
    filtro = cumple_tiempos_ciudad
    
    fig_cumple_tiempos_ciudad = go.Figure(data = [go.Table(header = dict(values = ["Ciudad","Cumplimiento UM","Cumplimiento Tiempo Total"]),
                                                       cells = dict(values = [filtro["Ciudad"],filtro["CumplimientoUM"], filtro["CumplimientoTiempoTotal"]]))])
  
    fig_cumple_tiempos_ciudad.update_layout(title = "Cumplimiento tiempos por ciudad")

    return fig_cumple_tiempos_ciudad


@app.callback(
    Output("TiempoAsignacion", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_TiempoAsignacion(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro =  pivot_asignacion_p[pivot_asignacion_p["Proveedor"]==value]
        
    else:   
        
        filtro =  pivot_asignacion[pivot_asignacion["Proveedor"]==value]
        filtro =  filtro[filtro["SHIPNODE_KEY"]==valuepdv]
    
    fig_asignacion = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Asignacion (min)"],mode="lines+markers+text",text=filtro["Tiempo Asignacion (min)"],textposition="bottom center")])
    fig_asignacion.update_layout(title = "Tiempo promedio asignación", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    
    return fig_asignacion


@app.callback(
    Output("CumplimientoAsignacion", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv", "value"))
def update_CumpleAsignacion(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_cumple_asignacion_p[pivot_cumple_asignacion_p["Proveedor"]==value]
    
    else:
        filtro = pivot_cumple_asignacion[pivot_cumple_asignacion["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
    
    fig_cumple_asignacion = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Cumplimiento"],mode="lines+markers+text",text=filtro["Cumplimiento"],textposition="bottom center")])
    fig_cumple_asignacion.update_layout(title = "Cumplimiento asignación", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    
    return fig_cumple_asignacion


@app.callback(
    Output("TiempoRecogida", "figure"),
    Input("dropdown_prov","value"),
    Input("dropdown_pdv", "value"))
def update_TiempoRecogida(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro =  pivot_recogida_p[pivot_recogida_p["Proveedor"]==value]
        
    else:
        filtro =  pivot_recogida[pivot_recogida["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    fig_recogida = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Recogida (min)"],mode="lines+markers+text",text=filtro["Tiempo Recogida (min)"],textposition="bottom center",)])
    fig_recogida.update_layout(title = "Tiempo promedio recogida", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    
    return fig_recogida

@app.callback(
    Output("CumplimientoRecogida", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_CumpleRecogida(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_cumple_recogida_p[pivot_cumple_recogida_p["Proveedor"]==value]
        
    else:
        filtro = pivot_cumple_recogida[pivot_cumple_recogida["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    
    fig_cumple_recogida = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Cumplimiento"],mode="lines+markers+text",text=filtro["Cumplimiento"],textposition="bottom center")])
    fig_cumple_recogida.update_layout(title = "Cumplimiento recogida", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    
    return fig_cumple_recogida


@app.callback(
    Output("TiempoPunto", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_TiempoPunto(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro =  pivot_tiempo_punto_p[pivot_tiempo_punto_p["Proveedor"]==value]
        
    else:
        filtro =  pivot_tiempo_punto[pivot_tiempo_punto["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
    
    
    fig_punto = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Punto (min)"],mode="lines+markers+text",text=filtro["Tiempo Punto (min)"],textposition="bottom center")])
    fig_punto.update_layout(title = "Tiempo promedio en punto", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    return fig_punto

@app.callback(
    Output("CumplimientoTiempoPunto", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_CumplePunto(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_cumple_tiempo_punto_p[pivot_cumple_tiempo_punto_p["Proveedor"]==value]
        
    else:
        filtro = pivot_cumple_tiempo_punto[pivot_cumple_tiempo_punto["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    
    fig_cumple_punto =  go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Cumplimiento"],mode="lines+markers+text",text=filtro["Cumplimiento"],textposition="bottom center")])
    fig_cumple_punto.update_layout(title = "Cumplimiento tiempo en punto", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    
    return fig_cumple_punto

@app.callback(
    Output("TiempoRuta", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_TiempoRuta(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_tiempo_ruta_p[pivot_tiempo_ruta_p["Proveedor"]==value]
        
    else:
        filtro = pivot_tiempo_ruta[pivot_tiempo_ruta["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        

    fig_ruta =  go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Ruta (min)"],mode="lines+markers+text",text=filtro["Tiempo Ruta (min)"],textposition="bottom center")])
    fig_ruta.update_layout(title = "Tiempo promedio en ruta", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    
    return fig_ruta

@app.callback(
    Output("CumplimientoTiempoRuta", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_CumpleRuta(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_cumple_ruta_p[pivot_cumple_ruta_p["Proveedor"]==value]
        
    else: 
        filtro = pivot_cumple_ruta[pivot_cumple_ruta["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]   
        
    fig_cumple_ruta = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Cumplimiento"],mode="lines+markers+text",text=filtro["Cumplimiento"],textposition="bottom center")])
    fig_cumple_ruta.update_layout(title = "Cumplimiento tiempo en ruta", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    
    return fig_cumple_ruta

@app.callback(
    Output("TiempoTotal", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_TiempoTotal(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_tiempo_total_p[pivot_tiempo_total_p["Proveedor"]==value]
    
    else:
        filtro = pivot_tiempo_total[pivot_tiempo_total["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    
    fig_total = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Total (min)"],mode="lines+markers+text",text=filtro["Tiempo Total (min)"],textposition="bottom center")])
    fig_total.update_layout(title = "Tiempo promedio UM", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    return fig_total


@app.callback(
    Output("CumplimientoTiempoTotal", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_CumpleTotal(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_cumple_total_p[pivot_cumple_total_p["Proveedor"]==value]
        
    else: 
        filtro = pivot_cumple_total[pivot_cumple_total["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
    
    fig_cumple_total = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Cumplimiento"],mode="lines+markers+text",text=filtro["Cumplimiento"],textposition="bottom center")])
    fig_cumple_total.update_layout(title = "Cumplimiento tiempo total UM", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    return fig_cumple_total



@app.callback(
    Output("demora_int","figure"),
    Input("dropdown_prov","value"))
def update_criticos_int(value):
    
    filtro = Top_packin_vs_iniciado[Top_packin_vs_iniciado["Proveedor"]==value]
    
    fig_demoras_int = go.Figure(data =[go.Table(header = dict(values = ["Orden","Sucursal","Packing Complete","Inicio 3pl","SLA cliente","Packin vs Inicio 3pl","Estado_UM"]),
                                               cells = dict(values = [filtro["numorden"],filtro["SHIPNODE_KEY"],filtro["Packing_Complete_COL"],filtro["iniciado"],filtro["EXTN_ORG_REQ_SHIP_DATE"], filtro["Packin_vs_inicio_3pl"],filtro["Estado_UM"]]))])
    
    fig_demoras_int.update_layout(title = "Ordenes con demora en integración")
    
    return fig_demoras_int


@app.callback(
    Output("TiempoIntegracion", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_TiempoIntegracion(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_dif_packin_iniciado_p[pivot_dif_packin_iniciado_p["Proveedor"]==value]
        
    else:
        filtro = pivot_dif_packin_iniciado[pivot_dif_packin_iniciado["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    fig_integracion = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Tiempo Integracion (min)"],mode="lines+markers+text",text=filtro["Tiempo Integracion (min)"],textposition="bottom center")])
    fig_integracion.update_layout(title = "Tiempo desde packing hasta inicio 3pl", xaxis_title = "Franja Horaria",yaxis_title = "Tiempo (min)")
    
    return fig_integracion


@app.callback(
    Output("Participacion", "figure"), 
    Input("dropdown_prov", "value"),
    Input("dropdown_pdv","value"))
def update_Participacion(value,valuepdv):
    
    if valuepdv == "Todos":
        filtro = pivot_participacion_p[pivot_participacion_p["Proveedor"]==value]
    
    else:
        filtro = pivot_participacion_[pivot_participacion_["Proveedor"]==value]
        filtro = filtro[filtro["SHIPNODE_KEY"]==valuepdv]
        
    fig_participacion = go.Figure(data = [go.Scatter(x=filtro["Hora_Servicio"],y=filtro["Participacion"],mode="lines+markers+text",text=filtro["Participacion"],textposition="bottom center")])
    fig_participacion.update_layout(title = "Porcentaje ordenes tiempo de intregación > 10min", xaxis_title = "Franja Horaria",yaxis_title = "(%)")
    
    return fig_participacion


@app.callback(
    Output("criticas_asig","figure"),
    Input("dropdown_prov","value"))
def update_criticas_asignacion(value):
    
    filtro = df_sin_asignar[df_sin_asignar["Proveedor"]==value]
    fig_crit_asig = go.Figure(data = [go.Table(header = dict(values=["Orden","km","Tiempo de asignacion"]),
                                              cells = dict(values = [filtro["numorden"],filtro["distancia_km"],filtro["TiempoLive"]]))])
    fig_crit_asig.update_layout(margin=dict(l=20, r=20, t=40, b=20),title ="Ordenes críticas sin asignar")
    
    return fig_crit_asig


@app.callback(
    Output("criticas_rec","figure"),
    Input("dropdown_prov","value"))
def update_criticas_recogida(value):
    
    filtro = df_sin_recoger[df_sin_recoger["Proveedor"]==value]
    fig_crit_rec = go.Figure(data = [go.Table(header = dict(values = ["Orden","Km","Tiempo de recogida"]),
                                            cells = dict(values = [filtro["numorden"],filtro["distancia_km"],filtro["TiempoLive"]]))])
    
    fig_crit_rec.update_layout(margin=dict(l=20, r=20, t=40, b=20),title ="Ordenes críticas sin recoger")
    
    return fig_crit_rec
    
@app.callback(
    Output("criticas_cierre","figure"),
    Input("dropdown_prov","value"))
def update_criticas_cierre(value):
    
    filtro = df_en_proceso[df_en_proceso["Proveedor"]==value]
    fig_crit_proc = go.Figure(data = [go.Table(header = dict(values = ["Orden","Km","Tiempo desde llegada a punto"]),
                                              cells = dict(values = [filtro["numorden"],filtro["distancia_km"],filtro["TiempoLive"]]))])
    
    fig_crit_proc.update_layout(margin=dict(l=20, r=20, t=40, b=20),title ="Ordenes críticas sin cerrar")
    
    return fig_crit_proc

@app.callback(
    Output("puntos_asignacion","figure"),
    Input("dropdown_prov","value"))
def update_puntos_asignacion(value):
    
    filtro = pivot_asignacion_punto[pivot_asignacion_punto["Proveedor"] ==value]
    fig_puntos_asig = go.Figure(data = [go.Table(header = dict(values = ["Sucursal","%Cumplimiento Asignación"]),
                                                cells = dict(values =[filtro["Sucursal"],filtro["Cumplimiento"]]))])
    
    fig_puntos_asig.update_layout(margin = dict(l=20, r=20, t=40, b=20),title ="Porcentaje Cumplimiento Asignación (PDV)")
    
    return fig_puntos_asig


@app.callback(
    Output("puntos_recogida","figure"),
    Input("dropdown_prov","value"))
def update_puntos_recogida(value):
    
    filtro = pivot_recogida_punto[pivot_recogida_punto["Proveedor"]==value]
    fig_puntos_rec = go.Figure(data = [go.Table(header = dict(values = ["Sucursal","%Cumplimiento Recogida"]),
                                               cells = dict(values = [filtro["Sucursal"],filtro["Cumplimiento"]]))])
    
    
    fig_puntos_rec.update_layout(margin = dict(l=20, r=20, t=40, b=20),title = "Porcentaje Cumplimiento Recogida (PDV)")
    
    return fig_puntos_rec


@app.callback(
    Output("puntos_punto","figure"),
    Input("dropdown_prov","value"))
def update_puntos_punto(value):
    
    filtro = pivot_punto_punto[pivot_punto_punto["Proveedor"]==value]
    fig_puntos_punto = go.Figure(data = [go.Table(header = dict(values = ["Sucursal","%Cumplimiento T en punto"]),
                                                 cells = dict(values =[filtro["Sucursal"],filtro["Cumplimiento"]]))])
    
    
    fig_puntos_punto.update_layout(margin = dict(l=20, r=20, t=40, b=20), title = "Porcentaje Cumplimiento Tiempo en punto (PDV)")
    
    return fig_puntos_punto

@app.callback(
    Output("puntos_ruta","figure"),
    Input("dropdown_prov","value"))
def update_puntos_ruta(value):
    
    filtro = pivot_ruta_punto[pivot_ruta_punto["Proveedor"]==value]
    fig_puntos_ruta = go.Figure(data=[go.Table(header = dict(values = ["Sucursal","%Cumplimiento Ruta"]),
                                              cells= dict(values =[filtro["Sucursal"],filtro["Cumplimiento"]]))])

    
    fig_puntos_ruta.update_layout(margin = dict(l=20, r=20, t=40, b=20),title ="Porcentaje Cumplimiento Tiempo en Ruta (PDV)")
    
    return fig_puntos_ruta


@app.callback(
    Output("puntos_total","figure"),
    Input("dropdown_prov","value"))
def update_puntos_total(value):
    
    filtro = pivot_total_punto[pivot_total_punto["Proveedor"]==value]
    fig_puntos_total = go.Figure(data=[go.Table(header = dict(values = ["Sucursal","%Cumplimiento Tiempo UM"]),
                                               cells = dict(values = [filtro["Sucursal"],filtro["Cumplimiento"]]))])

    fig_puntos_total.update_layout(margin = dict(l=20, r=20, t=40, b=20), title = "Porcentaje Cumplimiento Tiempo Total UM (PDV)")
    
    return fig_puntos_total


    
if __name__ == '__main__':
    app.run_server(debug=False)

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/2594190169.py:206: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/2594190169.py:221: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\NATHAL~1.GUT\AppData\Local\Temp/ipykernel_12932/2594190169.py:236: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Mar/2023 17:13:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:13:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02

127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2023 17:21:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [ ]:
df_tiempo_um.to_excel(r"C:\Users\nathalia.gutierrez\Desktop\SeguimientoUM\TiemposUMyCreacion.xlsx")

In [ ]:
df_tiempo_um.columns